In [1]:
from typing import Optional, Literal
from pydantic import BaseModel, Field

import os
import fitz
import pandas as pd
from llama_index.core.program import LLMTextCompletionProgram
from pydantic import BaseModel
from llama_index.llms.ollama import Ollama

from typing_extensions import Literal as LiteralExt 

from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import MarkdownTextSplitter, RecursiveCharacterTextSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore


from uuid import uuid4

import chromadb
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.tools import tool

from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub


In [6]:
prompt = hub.pull("hwchase17/react")


In [9]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [2]:
df = pd.read_csv("./data/rag-data.csv")

In [6]:
df["category"].unique()

array(['domestic-worker', 'labour-inspection', 'labour-disputes',
       'establishments', 'fees-and-guarantees', 'grievances',
       'alternative-end-of-service-benefits-system',
       'occupational-health-and-safety-and-labour-accommodation',
       'training-and-employment-of-students',
       'private-employment-agencies', 'wage-protection',
       'workpermit-and-contract', 'emiratisation', 'mohre-services',
       'uae-jobs', 'mohre-faq', 'uae-visa-emirates-id',
       'uae-passport-travel'], dtype=object)

In [ ]:
client = chromadb.PersistentClient(path="./chroma_store")
collection_name="wr-uae"
collection = client.get_or_create_collection(collection_name)
embed_model = OllamaEmbeddings(model="nomic-embed-text:v1.5")


vector_store = Chroma(
    collection_name=collection_name,
    embedding_function=embed_model,
    client=client
)

In [ ]:
Category = Literal[
    "domestic-worker", "labour-inspection", "labour-disputes", "establishments",
    "fees-and-guarantees", "grievances", "alternative-end-of-service-benefits-system",
    "occupational-health-and-safety-and-labour-accommodation",
    "training-and-employment-of-students", "private-employment-agencies",
    "wage-protection", "workpermit-and-contract", "emiratisation", "mohre-services",
    "uae-jobs", "mohre-faq", "uae-visa-emirates-id", "uae-passport-travel"
]

class FindRelevantArgs(BaseModel):
    query: str = Field(..., description="User query to search for semantically similar documents.")
    category: Optional[Category] = Field(
        None,
        description="Optional category filter; limits results to this category."
    )
    k: int = Field(
        5,
        ge=1, le=20,
        description="Number of top results to return (defaults to 5; max 20)."
    )
    
def cut(text: str, max_chars: int = 1200) -> str:
    if len(text) <= max_chars:
        return text
    return text[:max_chars] + " …[truncated]"

@tool("find_relevant", args_schema=FindRelevantArgs)
def find_relevant(query: str, category: str | None = None, k: int = 5) -> str:
    """Search the vector store and return a compact RESULTS list plus a CONTEXT block."""
    filt = {"category": category} if category else None
    hits = vector_store.similarity_search(query, k=k, filter=filt)

    lines = []
    ctx = []
    for i, d in enumerate(hits, 1):
        cat = (d.metadata or {}).get("category", "unknown")
        doc_id = (d.metadata or {}).get("id", d.id)
        lines.append(f"[{i}] id={doc_id} category={cat}")
        ctx.append(f"[{i}] {cut(d.page_content)}")

    return "RESULTS:\n" + "\n".join(lines) + "\n\nCONTEXT:\n" + "\n\n---\n\n".join(ctx)


class GetFullArgs(BaseModel):
    doc_id: str = Field(..., description="Document ID to fetch the full, untruncated text for.")

@tool("get_full", args_schema=GetFullArgs)
def get_full(doc_id: str) -> str:
    """Fetch the full text of a single document by its ID from the vector store."""
    try:
        docs = vector_store.get_by_ids([doc_id])
        return docs[0].page_content if docs else f"Not found: {doc_id}"
    except Exception as e:
        return f"Error fetching {doc_id}: {e}"